# Minimax

To implement a player, we are offered two functions: __move__ and __update__. A _Minimax_ player uses a _deterministic_ algorithm... the player always decides what to do based solely on the state of the game... so __update__ isn't really needed. Everything happens in __move__, because picking the next move, based on the state of the game, is everything. There's no machine learning needed.

In [ ]:
from tictactoe import *

# here is an empty Minimax player
class MinimaxPlayer(BasePlayer):
    
    # return a valid move (0..9, equal to [row * 3 + col] )
    def move(self, game, state):
        pass

    # receive feedback from most recent move
    def update(self, game, state, reward):
        pass
    

The __Game__ class provides support that we will use when building our _Minimax_ player, including:
- assigning each player an identifying number (1 or -1)
- providing the board (a 3x3 array containing 1,0,-1... the zeros are empty squares)
- an evaluation function to determine if a given state represents a win or loss
- a function to indicate when the game is over
    

In [ ]:
# Here is an empty game board -- note that the evaluation function g.max_min()
# returns a tuple of two values: (a,b). These are the maximum and minimum sums
# of any row, column, or diagonal, so a max of +3 means that the 'X' player has 
# won, and a min of -3 means that the 'O' player has won.

g = Game()
print('-----------------------------------------------------')
print(g)
print('-----------------------------------------------------')
print(g.board)
print('-----------------------------------------------------')
print('min_max=',g.max_min(),'game over?',g.game_over())

Let's replace the board with something that can be evaluated:

In [ ]:
# Inside the Game class, 'X' is represented as 1 and 'O' is -1;
# blank cells are zeros.. and the board must be a numpy array!... 
# but we won't be accessing the board directly, this is just a 
# hack to show how the system works.

g.board=np.array([[1,0,-1],[0,1,-1],[-1,0,1]]) 
print('-----------------------------------------------------')
print(g)
print('-----------------------------------------------------')
print(g.board)
print('-----------------------------------------------------')
print('min_max=',g.max_min(),'game over?',g.game_over())

So... if we can access the state of the board and the outcome of the game, the only thing remaining is to know which player is which. The __Game__ class distinguishes by assiging an n-value to each player at runtime. The n-value for 'X' is 1 and the n-value for 'O' is -1:

In [ ]:
x = MinimaxPlayer()
o = MinimaxPlayer()
g = Game(x,o)
print('x.n = ',x.n, '   o.n = ',o.n, '   max_min = ', g.max_min())

So outcomes work like this: if __Game.max_min__ returns player.n * 3, that means the board represents a win for player.n, keeping in mind that __Game.max_min__ returns a pair of values (max,min).

We also need to know if a game ends in a tie. That happens when the board is full, but no player has a score of 3.

Let's add __get_score__, __board_is_full__ and __minimax__ functions to our player:

In [ ]:
from tictactoe import *

# here is an empty Minimax player
class MinimaxPlayer(BasePlayer):
    
    # return a valid move (0..9, equal to [row * 3 + col] )
    def move(self, game, state):
        pass

    # receive feedback from most recent move
    def update(self, game, state, reward):
        pass
    
    # do the silly housekeeping of representing all scores as positive values (0..3)
    def get_score(self, game):
        max_min = game.max_min()
        if self.n == 1:
            return max_min[0]   # use the maximum when playing 'X'
        else:
            return -max_min[1]  # flip the minimum when playing 'O'
    
    # check if the board is full (implying that the game is over)
    def board_is_full(self, board):
        for row in range(3):
            for col in range(3):
                if board[row][col] == 0:
                    return False
        return True
    
    # do the real work first described by Jon Von Neumann in 1928
    def minimax(self, game, isMe):    
        
        # first, check for terminal conditions...
        score = self.get_score(game)
        if abs(score) == 3:
            return score     # a winner or loser... either is a terminal state
        if self.board_is_full(game.board):
            return 0         # a tie is also a terminal state
        
        if isMe:
            best = -4
            for row in range(3):
                for col in range(3):
                    if g.board[row][col] == 0:
                        g.board[row][col] = self.n
                        best = max(best, self.minimax(game, not isMe))
                        g.board[row][col] = 0
        else:
            best = 4
            for row in range(3):
                for col in range(3):
                    if g.board[row][col] == 0:
                        g.board[row][col] = -self.n
                        best = min(best, self.minimax(game, not isMe))
                        g.board[row][col] = 0
        return best

In [ ]:
x = MinimaxPlayer()
o = MinimaxPlayer()
g = Game(x,o)
# according to Minimax theory, X can win this board but O cannot:
g.board = np.array([[0,1,0],[-1,0,1],[1,-1,-1]])
print(g)
print('X Player minimax = ', x.minimax(g,True))
print('O Player minimax = ', o.minimax(g,True))

Why can X still win the game, but O cannot?

Minimax assumes that either player will make his optimum move, so if it is X's turn:
X = 2
O = 0 (to prevent a loss)
X = 4 (winning, on the diagonal)

If it is O's turn, no move forces X into any inferior positon, so the best outcome is a tie.

The next step is to use __minimax__ to choose an actual move:

In [1]:
from tictactoe import *

# here is an empty Minimax player
class MinimaxPlayer(BasePlayer):
    
    # return a valid move (0..9, equal to [row * 3 + col] )
    def move(self, game, state):
        best_row = -1
        best_col = -1
        best_score = -100
        for row in range(3):
            for col in range(3):
                if game.board[row][col] == 0:
                    game.board[row][col] = self.n
                    score = self.minimax(game,True)
                    print(row,col,row*3+col,score)
                    game.board[row][col] = 0
                    if score > best_score:
                        best_score = score
                        best_row = row
                        best_col = col
        return best_row * 3 + best_col

    # receive feedback from most recent move
    def update(self, game, state, reward):
        if reward == -1:
            print('Inconceivable!')  # how did we manage to lose, having examined every outcome?
    
    # do the silly housekeeping of representing all scores as positive values (0..3)
    def get_score(self, game):
        max_min = game.max_min()
        if self.n == 1:
            if max_min[0] == 3:
                return 1
            elif max_min[1] == -3:
                return -1
            else:
                return 0
        else:
            if max_min[1] == -3:
                return 1
            elif max_min[0] == 3:
                return -1
            else:
                return 0
    
    # check if the board is full (implying that the game is over)
    def board_is_full(self, board):
        for row in range(3):
            for col in range(3):
                if board[row][col] == 0:
                    return False
        return True
    
    # do the work first described by Jon Von Neumann in 1928
    def minimax(self, game, isMe):    
        
        # first, check for terminal conditions...
        score = self.get_score(game)
        if abs(score) == 3:
            return score     # a winner or loser... either is a terminal state
        if self.board_is_full(game.board):
            return 0         # a tie is also a terminal state
        
        if isMe:
            best = -4
            for row in range(3):
                for col in range(3):
                    if g.board[row][col] == 0:
                        g.board[row][col] = self.n
                        best = max(best, self.minimax(game, not isMe))
                        g.board[row][col] = 0
        else:
            best = 4
            for row in range(3):
                for col in range(3):
                    if g.board[row][col] == 0:
                        g.board[row][col] = -self.n
                        best = min(best, self.minimax(game, not isMe))
                        g.board[row][col] = 0
        return best

In [5]:
from tictactoe import *
x = MinimaxPlayer()
o = MinimaxPlayer()
g = Game(x,o)
# according to Minimax theory, X can win this board but O cannot:
g.board = np.array([[0,1,0],[-1,0,1],[1,-1,-1]])
print(g)
print('X Player minimax = ', x.minimax(g,True),' move = ', x.move(g,0))
print('O Player minimax = ', o.minimax(g,True),' move = ', o.move(g,0))


---( 0 )--------------------------

      \ /         
 0     X     2    
      / \         

OOO         \ /   
O O    4     X    
OOO         / \   

\ /   OOO   OOO   
 X    O O   O O   
/ \   OOO   OOO   

x state = 2041, o state = 17641
0 0 0 0
0 2 2 0
1 1 4 0
X Player minimax =  0  move =  0
0 0 0 0
0 2 2 0
1 1 4 0
O Player minimax =  0  move =  0


In [3]:
x = MinimaxPlayer()
o = HumanPlayer()
g = Game(x,o)
g.board = np.array([[-1,0,1],[1,0,-1],[-1,1,0]])
print(g)
print(x.move(g,0))


---( 0 )--------------------------

OOO         \ /   
O O    1     X    
OOO         / \   

\ /         OOO   
 X     4    O O   
/ \         OOO   

OOO   \ /         
O O    X     8    
OOO   / \         

x state = 11091, o state = 8591
0 1 1 0
1 1 4 0
2 2 8 0
1


In [ ]:
x = MinimaxPlayer()
o = HumanPlayer()
g = Game(x,o)
g.play()

0 0 0 0
0 1 1 0
0 2 2 0
1 0 3 0
1 1 4 0
1 2 5 0
2 0 6 0
2 1 7 0
2 2 8 0

---( 1 )--------------------------

\ /               
 X     1     2    
/ \               

                  
 3     4     5    
                  

                  
 6     7     8    
                  

x state = 9842, o state = 9840
8
0 1 1 0
0 2 2 0
1 0 3 0
1 1 4 0
1 2 5 0
2 0 6 0
2 1 7 0

---( 3 )--------------------------

\ /   \ /         
 X     X     2    
/ \   / \         

                  
 3     4     5    
                  

            OOO   
 6     7    O O   
            OOO   

x state = 3284, o state = 16398
2
1 0 3 0
1 1 4 0
1 2 5 0
2 0 6 0
2 1 7 0

---( 5 )--------------------------

\ /   \ /   OOO   
 X     X    O O   
/ \   / \   OOO   

\ /               
 X     4     5    
/ \               

            OOO   
 6     7    O O   
            OOO   

x state = 3302, o state = 16380
